# sweep

> Set of functions for querying an area of OLC grids

In [ ]:
#| default_exp sweep

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
import pandas as pd
from openlocationcode import openlocationcode as olc
from olc_utils.grid import *
from typing import Tuple

> Find all grids within the area defined by two OLC grids as vertices

Given a string of characters representing X or Y characters of two OLC grids, find the characters of max and min grid

In [ ]:
# | export
def get_max_min_chars(
    str1: str,
    str2: str,
) -> Tuple[str, str]:
    max_str = ""
    min_str = ""
    for i in range(len(str1)):
        if str1[i] == str2[i]:
            max_str += str1[i]
            min_str += str1[i]

        elif olc.CODE_ALPHABET_.index(str1[i]) > olc.CODE_ALPHABET_.index(str2[i]):
            max_str += str1[i:]
            min_str += str2[i:]
            break
        else:
            max_str += str2[i:]
            min_str += str1[i:]
            break

    return max_str, min_str

In [ ]:
test_eq(get_max_min_chars("856X","8W6V"), ('8W6V', '856X'))
test_eq(get_max_min_chars("8W44","8W3X"), ('8W44', '8W3X'))

Given two OLC grids, find the grids that are Lower Left and Upper Right vertices in the rectangle defined by the two grids

In [ ]:
# | export
def get_right_diagonal_vertex_grids(
    grid1: str,
    grid2: str,
)-> Tuple[str, str]:
    if len(grid1) != len(grid2):
        raise ValueError("Grids must be of the same length")

    # Get the characters for X axis
    x_grid_1 = ''.join([x for i, x in enumerate(grid1) if i % 2 == 1])
    x_grid_2 = ''.join([x for i, x in enumerate(grid2) if i % 2 == 1])

    # Get the characters for Y axis
    y_grid_1 = ''.join([x for i, x in enumerate(grid1) if i % 2 == 0])
    y_grid_2 = ''.join([x for i, x in enumerate(grid2) if i % 2 == 0])
    
    # Calculate the X max & min characters
    x_max_grid, x_min_grid = get_max_min_chars(x_grid_1, x_grid_2)
    y_max_grid, y_min_grid = get_max_min_chars(y_grid_1, y_grid_2)
    
    left_bottom = ''.join(''.join(tup) for tup in zip(y_min_grid, x_min_grid))
    right_top =  ''.join(''.join(tup) for tup in zip(y_max_grid, x_max_grid))
    
    return left_bottom, right_top

In [ ]:
test_eq(get_right_diagonal_vertex_grids("8FWF6MX2", "8FWF6MV5"), ('8FWF6MV2', '8FWF6MX5'))
test_eq(get_right_diagonal_vertex_grids("8FWG424W", "8FWG4344"), ("8FWG424W", "8FWG4344"))
test_eq(get_right_diagonal_vertex_grids("8FWG422W", "8FWG424W"), ("8FWG422W", "8FWG424W"))

Given the right diagonal vextices, get the character ranges for X and Y sweeps

In [ ]:
# TODO
def get_XY_char_sweeps(
    grid1: str,
    grid2: str,
) -> Tuple[list]:
    pass

In [ ]:
get_XY_char_sweeps('8FWF6MV2', '8FWF6MX5')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()